In [97]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from typing import List
import pandas as pd
import sacrebleu
from rouge_score import rouge_scorer
import evaluate
rouge = evaluate.load('rouge')
sacrebleu = evaluate.load("sacrebleu")
chrf = evaluate.load("chrf")

In [98]:
# d = pd.read_csv('./amharic_title_generation/gpt4_responses-amharic_title_generation.csv').input.values.tolist()


In [99]:

data = pd.read_csv('./Gpt-4/amharic_mt_eng-amh/gpt4_responses-amharic_mt eng-amh.csv')
# print(data.Datasource.value_counts())
data.head()

,instruction,input,ouptput,datasource,response
0,የተሰጠውን ጽሁፍ ከ English ቋንቋ ወደ አማርኛ ቋንቋ ተርጉም።,"""We now have 4-month-old mice that are non-dia...","""አሁን የስኳር በሽተኛ ያልነበሩ አሁን ግን የሆኑ የ4-ወር-ዕድሜ ያላቸው...",amharic_mt eng-amh,"""አሁን 4 ወር ያህል አድርጏ ያሉ እርኳኖች አሁን የስኳር በሽታ የሌላቸው..."
1,የተሰጠውን ጽሁፍ ከ English ቋንቋ ወደ አማርኛ ቋንቋ ተርጉም።,"Dr. Ehud Ur, professor of medicine at Dalhousi...",ዶክተር ኢሁድ ኡር፣ በሃሊፉሽ፣ ኖቫስኮሽያ ውስጥ በሚገኘው በዳልሃውዚ ዩኒ...,amharic_mt eng-amh,''' ዶ/ር ኡሁድ ኡር፣ የህክምና ፕሮፌሰር በዳሎሃውሲ ዩኒቨርሲቲ በሀልፋ...
2,የተሰጠውን ጽሁፍ ከ English ቋንቋ ወደ አማርኛ ቋንቋ ተርጉም።,"Like some other experts, he is skeptical about...",ልክ እንደ ሌሎች ባለሙያዎች፣ አይነት 1 የስኳር በሽታ ላለባቸው ሰዎች እ...,amharic_mt eng-amh,"'''እንደ ሌሎች ጠቀምታዎችም እሱ, ዲያቤቴስ መፈወስ እንደሚቻል በአማልጅ..."
3,የተሰጠውን ጽሁፍ ከ English ቋንቋ ወደ አማርኛ ቋንቋ ተርጉም።,"On Monday, Sara Danius, permanent secretary of...",ሰኞ ዕለት፣ ሳራ ዳኒየስ፣ ለኖቤል ኮሚቴ ለስነ ጽሁፍ በስዊድናዊ አካዳሚ ...,amharic_mt eng-amh,በሰኞ፡ ሠራ ዳኒውስ፣ በስዊድነሽ አካዳሚ የኖቤል የቅርም ወጪ ማህበር የዘ...
4,የተሰጠውን ጽሁፍ ከ English ቋንቋ ወደ አማርኛ ቋንቋ ተርጉም።,"Danius said, ""Right now we are doing nothing. ...",“አሁን ምንም እያደግን አይደለም። ለቅርብ ተባባሪው ደውያለሁ አና ኢሜይል...,amharic_mt eng-amh,"ዳኒዩስ አለ፣ ""በአሁኑ ሰዓት ባለሙያው ምንም አይሰራም። ለቀረበው በተቀማ..."


In [100]:
# data=data[data.Datasource=="amharic_mt amh-eng"]
# data.drop_duplicates(subset='Input Text', keep="last")
# data = data.head(1300)
data = data.fillna('')
data.tail()

,instruction,input,ouptput,datasource,response
1007,የተሰጠውን ጽሁፍ ከ English ቋንቋ ወደ አማርኛ ቋንቋ ተርጉም።,"As the areas are sparsely populated, and light...",አከባቢዎቹ ጥቂት የሰፈሩ ሰዎች ስላሉበት፣ እና የብርሃን ብክለት ብዙውን ...,amharic_mt eng-amh,'''እንደሚመጡት የምድር ቦታዎች በቀላሉ ሰፊ ሰፊ በመፈሰጋቸው ምክንያት፣...
1008,የተሰጠውን ጽሁፍ ከ English ቋንቋ ወደ አማርኛ ቋንቋ ተርጉም።,Japanese work culture is more hierarchical and...,የጃፓን የሥራ ባህል ምዕራባውያን ከለመዱት ሁኔታ በተለየ መልኩ የበለጠ ተ...,amharic_mt eng-amh,'''የጃፓን የስራ ባህል ከምስራቅ ሰዎች ሊሰሩበት የሚችሉት ከሆነ የተበል...
1009,የተሰጠውን ጽሁፍ ከ English ቋንቋ ወደ አማርኛ ቋንቋ ተርጉም።,"Suits are standard business attire, and cowork...",ሱፍ መደበኛ የስራ ልብስ ነው እና ባልደረባዎች እርስ በርስ የሚጠራሩት በ...,amharic_mt eng-amh,'''እየተመረጡ ስብስቦች የስራ አደራሽ ቀሚስ ናቸው፤ እና የሥራ ባልንጀራ...
1010,የተሰጠውን ጽሁፍ ከ English ቋንቋ ወደ አማርኛ ቋንቋ ተርጉም።,"Workplace harmony is crucial, emphasizing grou...",የስራ ቦታ ስምምነት የግል ስኬቶችን ከማድነቅ ይልቅ የቡድን ጥረት ላይ ት...,amharic_mt eng-amh,'''የስራ ቦታ ስምምነት አስፈላጊ ነው፣ ከመድሃኒት የግለሰብ ድርሻዎች ጋ...
1011,የተሰጠውን ጽሁፍ ከ English ቋንቋ ወደ አማርኛ ቋንቋ ተርጉም።,Workers must often get their superiors' approv...,ሰራተኞች ለሚሰጡት ውሳኔዎች በሙሉ የበላዮቻቸውን በአብዝኛው ተቀባይነት ሊ...,amharic_mt eng-amh,'''ሠራተኞች በተለይ ለማድረጋቸው የሚኖሯቸውን ውሳኔዎች ከላይኛቸው ፋብ ...


In [101]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/israela/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [102]:
def calculate_bleu_score(references: List[List[str]], candidates: List[str]) -> float:
    scores = []
    for ref, cand in zip(references, candidates):
        candidate_tokens = nltk.word_tokenize(cand)
        reference_tokens = [nltk.word_tokenize(r) for r in ref]
        
        score = sentence_bleu(reference_tokens, candidate_tokens)
        scores.append(score)

    
    print(f" sentence bleu : {sum(scores) / len(scores) if scores else 0}")

    bleu = sacrebleu.compute(predictions=candidates, references=references)
    print(f"SacreBLEU score: {bleu}")

    chr = chrf.compute(predictions=candidates, references=references, word_order=2)
    print(f"corpus_chrf score: {chr}")

    results = rouge.compute(predictions=candidates, references=references)
    print(f"Rogue results: {results}")

   





In [103]:
calculate_bleu_score(data.ouptput.values.tolist(), data.response.values.tolist())


/Users/israela/anaconda3/envs/orca/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/israela/anaconda3/envs/orca/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/israela/anaconda3/envs/orca/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

 sentence bleu : 1.9426791601969513e-232
SacreBLEU score: {'score': 2.4962305976408814, 'counts': [3569, 765, 217, 66], 'totals': [19361, 18349, 17345, 16344], 'precisions': [18.433965187748566, 4.169164532127091, 1.2510810031709427, 0.40381791483113066], 'bp': 1.0, 'sys_len': 19361, 'ref_len': 17752}
corpus_chrf score: {'score': 19.121691648315377, 'char_order': 6, 'word_order': 2, 'beta': 2}
Rogue results: {'rouge1': 0.1899549917583514, 'rouge2': 0.07502744839701361, 'rougeL': 0.18679193829885526, 'rougeLsum': 0.18644794215446386}
